In [58]:
from sqlalchemy import MetaData, create_engine, ForeignKey, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

engine = create_engine('sqlite:///corona_analysis.db', echo=True)
meta_data = MetaData(bind=engine)
Base = declarative_base()

# Agegroups

In [50]:
class Agegroups_05y(Base):
   __tablename__ = '_agegroups_05y'
   #__table_args__ = {'extend_existing': True}

   agegroups_05y_id = Column(Integer, primary_key = True)
   agegroup = Column(String, nullable=False, unique=True)

class Agegroups_10y(Base):
   __tablename__ = '_agegroups_10y'

   agegroups_10y_id = Column(Integer, primary_key = True)
   agegroup = Column(String, nullable=False, unique=True)
   number_observations = Column(Integer)
   avg_age = Column(Float)

/tmp/ipykernel_3623/1310360661.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Agegroups_05y, and will be replaced in the string-lookup table.
  class Agegroups_05y(Base):
/tmp/ipykernel_3623/1310360661.py:8: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Agegroups_10y, and will be replaced in the string-lookup table.
  class Agegroups_10y(Base):


In [25]:
def add_new_agegroup_05y(session: sessionmaker, agegroup: str):
    """
    Adds a new agegroup with a 5-year-interval
    """
    
    # check if agegroup already exist
    new_agegroup = (
        session.query(Agegroups_05y)
        .filter(Agegroups_05y.agegroup == agegroup)
    ).one_or_none()

    if new_agegroup is not None:
        return
    
    # create new agegroup
    new_agegroup = Agegroups_05y(agegroup=agegroup)
    
    # write to DB
    session.add(new_agegroup)
    session.commit()

In [18]:
Base.metadata.create_all(engine)

2022-07-01 23:33:05,709 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-01 23:33:05,710 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_agegroups_05y")
2022-07-01 23:33:05,710 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-01 23:33:05,711 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("_agegroups_05y")
2022-07-01 23:33:05,711 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-01 23:33:05,712 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_agegroups_10y")
2022-07-01 23:33:05,712 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-01 23:33:05,713 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("_agegroups_10y")
2022-07-01 23:33:05,713 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-01 23:33:05,713 INFO sqlalchemy.engine.Engine 
CREATE TABLE _agegroups_05y (
	agegroups_05y_id INTEGER NOT NULL, 
	agegroup VARCHAR NOT NULL, 
	PRIMARY KEY (agegroups_05y_id), 
	UNIQUE (agegroup)
)


2022-07-01 23:33:05,714 INFO sqlalchemy.engine.Engine [no key 0.00032s] (

In [28]:
# create session object which will be used for connection
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [30]:
agegroups_05y = [
    "00-04",
    "05-09",
    "10-14",
    "15-19",
    "20-24",
    "25-29",
    "30-34",
    "35-39",
    "40-44",
    "45-49",
    "50-54",
    "55-59",
    "60-64",
    "65-69",
    "70-74",
    "75-79",
    "80+",
    "UNK"
]
for item in agegroups_05y:
    add_new_agegroup_05y(session=session, agegroup=item)

2022-07-01 23:44:19,248 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-01 23:44:19,249 INFO sqlalchemy.engine.Engine SELECT _agegroups_05y.agegroups_05y_id AS _agegroups_05y_agegroups_05y_id, _agegroups_05y.agegroup AS _agegroups_05y_agegroup 
FROM _agegroups_05y 
WHERE _agegroups_05y.agegroup = ?
2022-07-01 23:44:19,250 INFO sqlalchemy.engine.Engine [cached since 505.6s ago] ('00-04',)
2022-07-01 23:44:19,251 INFO sqlalchemy.engine.Engine SELECT _agegroups_05y.agegroups_05y_id AS _agegroups_05y_agegroups_05y_id, _agegroups_05y.agegroup AS _agegroups_05y_agegroup 
FROM _agegroups_05y 
WHERE _agegroups_05y.agegroup = ?
2022-07-01 23:44:19,251 INFO sqlalchemy.engine.Engine [cached since 505.6s ago] ('05-09',)
2022-07-01 23:44:19,252 INFO sqlalchemy.engine.Engine SELECT _agegroups_05y.agegroups_05y_id AS _agegroups_05y_agegroups_05y_id, _agegroups_05y.agegroup AS _agegroups_05y_agegroup 
FROM _agegroups_05y 
WHERE _agegroups_05y.agegroup = ?
2022-07-01 23:44:19,252 INFO sqlalchemy

In [34]:
def add_new_agegroup_10y(session: sessionmaker, agegroup: str, number_observations: int, avg_age: float):
    """
    Adds a new agegroup with a 10-year-interval
    """
    
    # check if agegroup already exist
    new_agegroup = (
        session.query(Agegroups_10y)
        .filter(Agegroups_10y.agegroup == agegroup)
    ).one_or_none()

    if new_agegroup is not None:
        return
    
    # create new agegroup
    new_agegroup = Agegroups_10y(
        agegroup=agegroup, 
        number_observations=number_observations, 
        avg_age=avg_age
    )
    
    # write to DB
    session.add(new_agegroup)
    session.commit()

In [35]:
agegroups_10y = { # agegroup, [number_observations, avg_age]
    "00-09": [10, 4.5],
    "10-19": [10, 14.5],
    "20-29": [10, 24.5],
    "30-39": [10, 34.5],
    "40-49": [10, 44.5],
    "50-59": [10, 54.5],
    "60-69": [10, 64.5],
    "70-79": [10, 74.5],
    "80+": [21, 90],
    "UNK": [0, 0],
}
for k, v, in agegroups_10y.items():
    add_new_agegroup_10y(session=session, agegroup=k, number_observations=v[0], avg_age=v[1])

2022-07-01 23:51:02,318 INFO sqlalchemy.engine.Engine SELECT _agegroups_10y.agegroups_10y_id AS _agegroups_10y_agegroups_10y_id, _agegroups_10y.agegroup AS _agegroups_10y_agegroup, _agegroups_10y.number_observations AS _agegroups_10y_number_observations, _agegroups_10y.avg_age AS _agegroups_10y_avg_age 
FROM _agegroups_10y 
WHERE _agegroups_10y.agegroup = ?
2022-07-01 23:51:02,320 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ('00-09',)
2022-07-01 23:51:02,321 INFO sqlalchemy.engine.Engine INSERT INTO _agegroups_10y (agegroup, number_observations, avg_age) VALUES (?, ?, ?)
2022-07-01 23:51:02,321 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ('00-09', 10, 4.5)
2022-07-01 23:51:02,322 INFO sqlalchemy.engine.Engine COMMIT
2022-07-01 23:51:02,324 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-01 23:51:02,325 INFO sqlalchemy.engine.Engine SELECT _agegroups_10y.agegroups_10y_id AS _agegroups_10y_agegroups_10y_id, _agegroups_10y.agegroup AS _agegroups_10y_agegroup, _a

In [39]:
def get_agegroups_05y(session: sessionmaker):
    """Get a list of agegroup objects with a 05-year interval"""
    return session.query(Agegroups_05y).order_by(Agegroups_05y.agegroup).all()

def get_agegroups_10y(session: sessionmaker):
    """Get a list of agegroup objects with a 10-year interval"""
    return session.query(Agegroups_10y).order_by(Agegroups_10y.agegroup).all()

2022-07-01 23:55:21,805 INFO sqlalchemy.engine.Engine SELECT _agegroups_10y.agegroups_10y_id AS _agegroups_10y_agegroups_10y_id, _agegroups_10y.agegroup AS _agegroups_10y_agegroup, _agegroups_10y.number_observations AS _agegroups_10y_number_observations, _agegroups_10y.avg_age AS _agegroups_10y_avg_age 
FROM _agegroups_10y ORDER BY _agegroups_10y.agegroup
2022-07-01 23:55:21,806 INFO sqlalchemy.engine.Engine [cached since 43.91s ago] ()
00-09
10-19
20-29
30-39
40-49
50-59
60-69
70-79
80+
UNK


# Calendar

In [96]:
engine = create_engine('sqlite:///corona_analysis.db', echo=True)
meta_data = MetaData(bind=engine)
Base = declarative_base()

In [97]:
class CalendarYears(Base):
    __tablename__ = "_calendar_years"

    calendar_years_id = Column(Integer, primary_key=True)
    iso_year = Column(Integer, nullable=False, unique=True)
    calendar_weeks = relationship("CalendarWeeks", backref="_calendar_years")

In [98]:
class CalendarWeeks(Base):
    __tablename__ = "_calendar_weeks"

    calendar_weeks_id = Column(Integer, primary_key=True)
    calendar_years_fk = Column(Integer, ForeignKey("_calendar_years.calendar_years_id"), nullable=False)
    iso_week = Column(Integer, nullable=False)
    iso_key = Column(Integer, nullable=False, unique=True)
    calendar_years = relationship("CalendarDays", backref="_calendar_weeks")

In [99]:
class CalendarDays(Base):
    __tablename__ = "_calendar_days"

    calendar_days_id = Column(Integer, primary_key=True)
    calendar_weeks_fk = Column(Integer, ForeignKey("_calendar_weeks.calendar_weeks_id"), nullable=False)
    iso_day = Column(String, nullable=False, unique=True)

In [100]:
Base.metadata.create_all(engine)

2022-07-02 00:56:00,975 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-02 00:56:00,976 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_calendar_years")
2022-07-02 00:56:00,976 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-02 00:56:00,977 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("_calendar_years")
2022-07-02 00:56:00,977 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-02 00:56:00,978 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_calendar_weeks")
2022-07-02 00:56:00,978 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-02 00:56:00,979 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("_calendar_weeks")
2022-07-02 00:56:00,979 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-02 00:56:00,980 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_calendar_days")
2022-07-02 00:56:00,980 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-02 00:56:00,980 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("_calendar_days")
2022-07-02 00:56:00,

In [101]:
def add_new_calendar_year(session: sessionmaker, calendar_year: int) -> None:
    """
    Adds a new calendar year
    """
    
    # check if calendar year already exist
    new_calendar_year = (
        session.query(CalendarYears)
        .filter(CalendarYears.iso_year == calendar_year)
    ).one_or_none()

    if new_calendar_year is not None:
        return
    
    # create new agegroup
    new_calendar_year = CalendarYears(
        iso_year=calendar_year
    )
    
    # write to DB
    session.add(new_calendar_year)
    session.commit()

In [102]:
for year in range(1990, 2050):
    add_new_calendar_year(session=session, calendar_year=year)

2022-07-02 00:56:03,431 INFO sqlalchemy.engine.Engine SELECT _calendar_years.calendar_years_id AS _calendar_years_calendar_years_id, _calendar_years.iso_year AS _calendar_years_iso_year 
FROM _calendar_years 
WHERE _calendar_years.iso_year = ?
2022-07-02 00:56:03,433 INFO sqlalchemy.engine.Engine [generated in 0.00174s] (1990,)
2022-07-02 00:56:03,434 INFO sqlalchemy.engine.Engine INSERT INTO _calendar_years (iso_year) VALUES (?)
2022-07-02 00:56:03,435 INFO sqlalchemy.engine.Engine [generated in 0.00051s] (1990,)
2022-07-02 00:56:03,436 INFO sqlalchemy.engine.Engine COMMIT
2022-07-02 00:56:03,437 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-02 00:56:03,438 INFO sqlalchemy.engine.Engine SELECT _calendar_years.calendar_years_id AS _calendar_years_calendar_years_id, _calendar_years.iso_year AS _calendar_years_iso_year 
FROM _calendar_years 
WHERE _calendar_years.iso_year = ?
2022-07-02 00:56:03,438 INFO sqlalchemy.engine.Engine [cached since 0.006951s ago] (1991,)
2022-07-02 00

In [95]:
#CalendarDays.__table__.drop(engine)
#CalendarWeeks.__table__.drop(engine)
#CalendarYears.__table__.drop(engine)

2022-07-02 00:55:32,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-02 00:55:32,329 INFO sqlalchemy.engine.Engine 
DROP TABLE _calendar_weeks
2022-07-02 00:55:32,329 INFO sqlalchemy.engine.Engine [no key 0.00053s] ()
2022-07-02 00:55:32,331 INFO sqlalchemy.engine.Engine COMMIT
2022-07-02 00:55:32,333 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-02 00:55:32,333 INFO sqlalchemy.engine.Engine 
DROP TABLE _calendar_years
2022-07-02 00:55:32,333 INFO sqlalchemy.engine.Engine [no key 0.00038s] ()
2022-07-02 00:55:32,335 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
class Invoice(Base):
   __tablename__ = 'invoices'
   
   id = Column(Integer, primary_key = True)
   custid = Column(Integer, ForeignKey('customers.id'))
   invno = Column(Integer)
   amount = Column(Integer)
   customer = relationship("Customer", back_populates = "invoices")

In [12]:
Customer.invoices = relationship("Invoice", order_by = Invoice.id, back_populates = "customer")
Base.metadata.create_all(engine)

2022-07-01 22:44:39,419 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-01 22:44:39,419 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customers")
2022-07-01 22:44:39,420 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-01 22:44:39,420 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customers")
2022-07-01 22:44:39,421 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-01 22:44:39,422 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("invoices")
2022-07-01 22:44:39,422 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-01 22:44:39,423 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("invoices")
2022-07-01 22:44:39,423 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-01 22:44:39,424 INFO sqlalchemy.engine.Engine 
CREATE TABLE customers (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	address VARCHAR, 
	email VARCHAR, 
	PRIMARY KEY (id)
)


2022-07-01 22:44:39,424 INFO sqlalchemy.engine.Engine [no key 0.00028s] ()
2022-07-01 22:44:39,427 INFO sqlalchemy.engine.Engi